In [28]:
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from PIL import Image

import torch
from torch import nn
import torch.optim as optim
import torchvision
from torchvision import transforms, models, datasets
#https://pytorch.org/docs/stable/torchvision/index.html
import imageio
import time
import warnings
import random
import sys
import copy
import json

### 1. 数据读取与预处理操作

In [3]:
data_dir = '../face_data/sex/'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

#### 制作好数据源:
- data_transforms中指定了所有图像预处理操作
- lmageFolder假设所有的文件按文件夹保存好，每个文件夹下面存贮同一类别的图片，文件夹的名字为分类的名字
- 因为上采样数据集为（256，256），所以在这里做了中心裁剪到（224，224）
- 因为上采样数据集样本量较小，故在这里做了数据增强，有：1、随机旋转，-45到45度之间随机旋转；2、随机水平翻转；3、随机垂直翻转；4、随机亮度、对比度、饱和度、色相

In [4]:
# 三通道
data_transforms = {
    'train': transforms.Compose([transforms.RandomRotation(45), #随机旋转，-45到45度之间随机旋转
        transforms.CenterCrop(224), # 从中心开始裁剪
        transforms.RandomHorizontalFlip(p=0.5), # 随机水平翻转 选择一个概率概率
        transforms.RandomVerticalFlip(p=0.5),   # 随机垂直翻转
        transforms.ColorJitter(brightness=0.2, contrast=0.1, saturation=0.1, hue=0.1),# 亮度、对比度、饱和度、色相
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])#均值，标准差
    ]),
    'valid': transforms.Compose([transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [18]:
# 一通道
data_transforms = {
    'train': transforms.Compose([transforms.Grayscale(1), # 将图像转换为单通道图像
        transforms.RandomRotation(45), #随机旋转，-45到45度之间随机旋转
        transforms.CenterCrop(224), # 从中心开始裁剪
        transforms.RandomHorizontalFlip(p=0.5), # 随机水平翻转 选择一个概率概率
        transforms.RandomVerticalFlip(p=0.5),   # 随机垂直翻转
        transforms.ColorJitter(brightness=0.2, contrast=0.1, saturation=0.1, hue=0.1),# 亮度、对比度、饱和度、色相
        transforms.ToTensor(),
        transforms.Normalize(0.485, 0.229)#均值，标准差 单通道只需一个值
    ]),
    'valid': transforms.Compose([transforms.Grayscale(1),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(0.485, 0.229)
    ]),
}

In [19]:
batch_size = 8

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'valid' ]}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'valid']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
class_names = image_datasets['train'].classes


In [20]:
print(image_datasets)
print(dataloaders)
print(dataset_sizes)
print(class_names)

{'train': Dataset ImageFolder
    Number of datapoints: 3422
    Root location: ../face_data/sex/train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               RandomRotation(degrees=[-45.0, 45.0], interpolation=nearest, expand=False, fill=0)
               CenterCrop(size=(224, 224))
               RandomHorizontalFlip(p=0.5)
               RandomVerticalFlip(p=0.5)
               ColorJitter(brightness=[0.8, 1.2], contrast=[0.9, 1.1], saturation=[0.9, 1.1], hue=[-0.1, 0.1])
               ToTensor()
               Normalize(mean=0.485, std=0.229)
           ), 'valid': Dataset ImageFolder
    Number of datapoints: 396
    Root location: ../face_data/sex/valid
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=0.485,

### 2. 加载Resnet模型进行迁移学习
- 直接用训练的好权重当做初始化参数


In [9]:
#是否用GPU训练
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu :
    print('CUDA is not available. Training on CPU ...')
else:
    print('CUDA is available! Training on GPU ...')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


CUDA is available! Training on GPU ...


In [11]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [23]:
model_ft = models.resnet50()
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [51]:
# 初始化model，修改resnet为自己想要的
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    model_ft = None
    input_size = 0

    if model_name == "resnet" :
        model_ft = models.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract) # 设置参数使用预训练模型
        model_ft.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False) # 灰度图通道为1
        num_ftrs = model_ft.fc.in_features # 得到resnet最后一层的输入为2048
        model_ft.fc = nn.Sequential(nn.Linear(num_ftrs, 512), nn.Linear(512, num_classes), nn.LogSoftmax (dim=1))

        input_size = 224
    return model_ft, input_size

#### 需要训练哪些层

In [52]:
model_name = 'resnet'
feature_extract = True # 是否用人家训练好的特征来做

model_ft, input_size = initialize_model(model_name, 2, feature_extract, use_pretrained=True)

#GPU计算
model_ft = model_ft.to(device)

#模型保存
filename = '../modle_save/checkpoint.pth'

#是否训练所有层
params_to_update = model_ft.parameters()
print ("Params to learn:")
if feature_extract:
    params_to_update = []
    for name, param in model_ft.named_parameters() :
        if param.requires_grad == True:
            params_to_update.append (param)
            print(" \t" , name)
else:
    for name, param in model_ft.named_parameters() :
        if param.requires_grad == True:
            print(" \t" , name)


Params to learn:
 	 conv1.weight
 	 fc.0.weight
 	 fc.0.bias
 	 fc.1.weight
 	 fc.1.bias


In [53]:
model_ft

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### 优化器设置

In [61]:
#优化器设置
optimizer_ft = optim.Adam (params_to_update, lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)#学习率每7个epoch衰减成原来的1/10
#最后一层已经LogSoftmax()了，所以不能nn. brossEntropyloss()来计算了，n.CrossEntropyloss()相当于logSoftmax()和nn.NLLoss()整合
criterion = nn.NLLLoss()

### 训练模块

In [60]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, filename=filename):
    since = time.time()
    best_acc = 0

    model.to(device)
    val_acc_history = []
    train_acc_history = []
    train_losses = []
    valid_losses = []
    LRs = [optimizer.param_groups[0]['lr']]
    
    best_model_wts = copy.deepcopy(model.state_dict())
                                   
    for epoch in range (num_epochs) :
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
    
        #训练和验证
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train() # 训练
            else:
                model.eval()  # 验证

            running_loss = 0.0
            running_corrects = 0

            #把数据都取个遍
            for inputs, labels in dataloaders[phase] :
                inputs = inputs.to(device)
                labels = labels.to(device)

                #清零
                optimizer.zero_grad()
                #只有训练的时候计算和更新梯度
                with torch.set_grad_enabled(phase == 'train') :
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    #训练阶段更新权重
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                #计算损失
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)


            time_elapsed = time.time() - since
            print('Time elapsed {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
            print('{} Loss: {:.4f} Acc: {:.4f} '.format(phase, epoch_loss, epoch_acc))


            #得到最好那次的模型
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                state = {
                    'state_dict' : model.state_dict(),
                    'best_acc': best_acc,
                    'optimizer' : optimizer.state_dict(),
                }
                torch.save(state, filename)  # 保存最优结果
            if phase == 'valid':
                val_acc_history.append(epoch_acc)
                valid_losses.append (epoch_loss)
                scheduler.step(epoch_loss)
            if phase == 'train':
                train_acc_history.append(epoch_acc)
                train_losses.append(epoch_loss)

        print('Optimizer learning rate : {:.7f}'.format(optimizer.param_groups[0]['lr']))
        LRs.append(optimizer.param_groups[0]['lr'])
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f} s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {: 4f} '.format(best_acc))
    
    #训练完后用最好的一次当做模型最终的结果
    model.load_state_dict(best_model_wts) # 读取最优结果
    return model, val_acc_history, train_acc_history, valid_losses, train_losses, LRs



### 开始训练，修改conv1和fc层参数

In [62]:
# 初始训练
epochs = 5
model_ft, val_ace_history, train_acc_history, valid_losses, train_losses, LRs = train_model(model_ft, dataloaders, criterion, optimizer_ft, num_epochs=epochs, filename=filename)


Epoch 0/4
----------
Time elapsed 0m 53s
train Loss: 0.6929 Acc: 0.5809 
Time elapsed 0m 56s
valid Loss: 0.6022 Acc: 0.6717 
Optimizer learning rate : 0.0001000

Epoch 1/4
----------
Time elapsed 1m 48s
train Loss: 0.6418 Acc: 0.6341 
Time elapsed 1m 50s
valid Loss: 0.7147 Acc: 0.5606 
Optimizer learning rate : 0.0001000

Epoch 2/4
----------
Time elapsed 2m 42s
train Loss: 0.6431 Acc: 0.6315 
Time elapsed 2m 45s
valid Loss: 0.7601 Acc: 0.4823 
Optimizer learning rate : 0.0001000

Epoch 3/4
----------
Time elapsed 3m 37s
train Loss: 0.6248 Acc: 0.6540 
Time elapsed 3m 39s
valid Loss: 0.5846 Acc: 0.6894 
Optimizer learning rate : 0.0001000

Epoch 4/4
----------
Time elapsed 4m 32s
train Loss: 0.6242 Acc: 0.6502 
Time elapsed 4m 34s
valid Loss: 0.5543 Acc: 0.6667 
Optimizer learning rate : 0.0001000

Training complete in 4m 34 s
Best val Acc:  0.689394 


In [59]:
torch.save(model_ft, '../modle_save/modle_all.pth')
# model = torch.load(PATH)

### 再次训练，更改全部预训练参数

In [ ]:
# 再继续训练所有的参数，学习率调小一点
optimizer = optim.Adam(params_to_update, lr=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# 损失函数
criterion = nn.NLLLoss()

In [ ]:
#Load the checkpoint

# checkpoint = torch.load(filename)best_acc = checkpoint['best_acc']
# model_ft.load_state_dict(checkpoint['state_dict'])
# optimizer.load_state_dict(checkpoint[' optimizer']
# # model_ft.class_to_idx = checkpoint['mapping']

In [ ]:
# 再次训练
filename = '../modle_save/checkpoint_again.pth'
epochs = 5
model_ft, val_ace_history, train_acc_history, valid_losses, train_losses, LRs = train_model(model_ft, dataloaders, criterion, optimizer, num_epochs=epochs, filename=filename)
torch.save(model_ft, '../modle_save/modle_all_again.pth')